## Pothole detection

Jacek Piszczek
Damodar Łepski


In [1]:
ROAD_IMAGE_DATA_FOLDER = 'Road_Image_Data'
POTHOLE_IMAGE_DATA_FOLDER = 'Pothole_Image_Data'


# __Prepare image names from folders__

In [34]:
from subprocess import check_output

def get_image_list(folder_name):
    images = check_output(['ls', folder_name]).decode('utf-8').split('\n')
    return list(filter(None, images))


nice_road_images = get_image_list(ROAD_IMAGE_DATA_FOLDER)
pothole_images = get_image_list(POTHOLE_IMAGE_DATA_FOLDER)

# Scale and HOG images

In [38]:
# import matplotlib.pyplot as plt
import pandas as pd
from skimage.feature import hog
from skimage import data, exposure
from skimage.io import imread
from skimage.transform import resize



def scale_and_hog(folder_name, image_names):
    
    images_list = []
    features_list = []
    for img_name in image_names:
        image = imread('{}/{}'.format(folder_name, img_name))
        image = resize(image, (480, 640))
        try:
            fd, hog_image = hog(image, orientations=8, pixels_per_cell=(8, 8), cells_per_block=(1, 1), visualize=True, multichannel=True)

            hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 5))
            images_list.append(hog_image_rescaled)
            features_list.append(fd)
        except:
            print('Error: {}'.format(img_name))
    print("{} done!".format(ROAD_IMAGE_DATA_FOLDER))
    return features_list, images_list





# image = imread('Pothole_Image_Data/16.jpg')
# image = resize(image, (480, 640))

# fd, hog_image = hog(image, orientations=8, pixels_per_cell=(8, 8),
#                     cells_per_block=(1, 1), visualize=True, multichannel=True)

# print(fd.shape)
# print(fd)
# print(type(fd))

# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True)

# ax1.axis('off')
# ax1.imshow(image, cmap=plt.cm.gray)
# ax1.set_title('Input image')

# hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 5))

# print('**********************')

# print(type(hog_image_rescaled))
# print(hog_image_rescaled)
# print(hog_image_rescaled.shape)

# ax2.axis('off')
# ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
# ax2.set_title('Histogram of Oriented Gradients')
# plt.show()

# Scale and HOG nice road

In [72]:
nice_fd, nice_img = scale_and_hog(ROAD_IMAGE_DATA_FOLDER, nice_road_images)

Road_Image_Data done!


# Scale and HOG Potholes

In [39]:
pothole_fd, pothole_img = scale_and_hog(POTHOLE_IMAGE_DATA_FOLDER, pothole_images)

Error: 5.jpg
Road_Image_Data done!


# Create labels and merge to one datasets

In [81]:
nice_labels = [0 for _ in range(len(nice_fd))]
pothole_labels = [1 for _ in range(len(pothole_fd))]
print('Labels for nice roads: {}'.format(len(nice_labels)))
print('Labels for potholes: {}'.format(len(pothole_labels)))

dataset_X = nice_fd + pothole_fd
dataset_y = nice_labels + pothole_labels

# Test image labeling
# 0-49 should be images of nice roads (0)
# Above are potholes (1)
no = 55
print('******************')
print(dataset_X[no], dataset_y[no])


Labels for nice roads: 50
Labels for potholes: 148
******************
[0.24372526 0.         0.49460775 ... 0.04441135 0.22166617 0.16697653] 1


# Prepare data frame

In [108]:
import numpy as np

labels =  np.array(dataset_y).reshape(len(dataset_y), 1)

dataset_X = np.array(dataset_X)
# print(dataset_y[1])
data_frame = np.hstack((dataset_X, labels))
np.random.shuffle(data_frame)

print(data_frame)

[[0.70510639 0.01156467 0.         ... 0.39634008 0.22900596 0.        ]
 [0.41482165 0.41482165 0.41482165 ... 0.37893953 0.37893953 0.        ]
 [0.40321636 0.18150558 0.40321636 ... 0.37888827 0.37888827 0.        ]
 ...
 [0.         0.         0.         ... 0.38401583 0.31972691 1.        ]
 [0.44608982 0.44608982 0.44608982 ... 0.40382898 0.17753813 0.        ]
 [0.41928647 0.0822111  0.27782134 ... 0.42604605 0.11399411 1.        ]]


# Divide data to train and test

In [109]:
from sklearn import svm
train_percent = 80
p = int(len(dataset_X)*train_percent/100)

x_train, x_test = data_frame[:p,:-1],  data_frame[p:,:-1]
y_train, y_test = data_frame[:p,-1:].ravel() , data_frame[p:,-1:].ravel()

clf = svm.SVC()
clf.fit(x_train,y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

# Run prediction of SVC Classifier

In [110]:
from sklearn.metrics import classification_report,accuracy_score
y_pred = clf.predict(x_test)
print("Accuracy: "+str(accuracy_score(y_test, y_pred)))
print('\n')
print(classification_report(y_test, y_pred))

Accuracy: 0.8


              precision    recall  f1-score   support

         0.0       0.67      0.22      0.33         9
         1.0       0.81      0.97      0.88        31

    accuracy                           0.80        40
   macro avg       0.74      0.59      0.61        40
weighted avg       0.78      0.80      0.76        40

